# **1. INSTALL AND IMPORT ALL LIBRARIES**

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.4 MB/s eta 0:00:00


In [2]:
import os
import tempfile

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
from typing import Dict, Text

from itertools import combinations

import gdown
import zipfile

## **a. Download the dataset from google drive link**

In [3]:
# URL download
url_dataset = 'https://drive.usercontent.google.com/download?id=1XmYsJkX7FzVfsGFtzCQHEI9Yer_rGIrR&export=download&authuser=2&confirm=t&uuid=155a6595-930f-45ee-826e-f9079e703c04&at=APZUnTW0VgZ5C-NgzMH8iSoHcWVx:1701430824990'

# Replace with the desired names for the downloaded zip files
name_dataset = 'dataset.zip'

# Download the zip files
gdown.download(url_dataset, name_dataset, quiet=False)

# Unzip the downloaded files
with zipfile.ZipFile(name_dataset, 'r') as zip_ref:
    zip_ref.extractall('dataset')  # Extracts into 'dataset' folder

print("Files have been downloaded and extracted.")

Downloading...
From: https://drive.usercontent.google.com/download?id=1XmYsJkX7FzVfsGFtzCQHEI9Yer_rGIrR&export=download&authuser=2&confirm=t&uuid=155a6595-930f-45ee-826e-f9079e703c04&at=APZUnTW0VgZ5C-NgzMH8iSoHcWVx:1701430824990
To: /content/dataset.zip
100%|██████████| 96.6k/96.6k [00:00<00:00, 20.2MB/s]

Files have been downloaded and extracted.


# **2. LOAD DATA**

## **a. Food Data**

In [4]:
food_data = pd.read_csv("dataset/dataset/food_data_final.csv")

In [5]:
food_data

,food_id,nama_makanan,sumber,tipe,jenis_olahan,kalori,protein,lemak,karbohidrat,daging_kerbau,...,daging_babi,daging_kambing,daging_sapi,ikan,kedelai,sayur,susu,telur_ayam,tepung,umbi_umbian
0,FNT001,Bakso,Daging Sapi,Makanan Berat,Rebus,202,12.41,13.16,7.58,0,...,0,0,1,0,0,0,0,0,0,0
1,FNT002,Bubur Ayam,"Beras, Daging Ayam",Makanan Berat,Rebus,155,11.48,5.16,15.05,0,...,0,0,0,0,0,0,0,0,0,0
2,FNT003,Mi Goreng,Tepung,Makanan Berat,Rebus,475,10.00,15.00,65.00,0,...,0,0,0,0,0,0,0,0,1,0
3,FNT004,Sate,"Daging Ayam, Daging Kambing",Makanan Berat,Bakar,225,19.54,14.82,4.87,0,...,0,1,0,0,0,0,0,0,0,0
4,FNT005,Soto,"Daging Ayam, Daging Sapi",Makanan Berat,Rebus,130,9.96,6.19,8.11,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,FNT147,Kue Sus,Tepung,Makanan Ringan,Kukus,221,7.50,10.20,24.80,0,...,0,0,0,0,0,0,0,0,1,0
147,FNT148,Martabak Mesir,"Tepung, Telur, Daging Ayam, Daging Sapi",Makanan Berat,Goreng,278,5.10,8.60,45.00,0,...,0,0,1,0,0,0,0,1,1,0
148,FNT149,Mie Aceh Rebus,"Tepung, Telur, Daging Ayam",Makanan Berat,Rebus,113,3.00,3.20,18.10,0,...,0,0,0,0,0,0,0,1,1,0
149,FNT150,Mie Pangsit Basah,"Tepung, Telur, Daging Ayam",Makanan Berat,Rebus,105,5.90,4.90,9.40,0,...,0,0,0,0,0,0,0,1,1,0


In [6]:
food_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   food_id         151 non-null    object 
 1   nama_makanan    151 non-null    object 
 2   sumber          151 non-null    object 
 3   tipe            151 non-null    object 
 4   jenis_olahan    151 non-null    object 
 5   kalori          151 non-null    int64  
 6   protein         151 non-null    float64
 7   lemak           151 non-null    float64
 8   karbohidrat     151 non-null    float64
 9   daging_kerbau   151 non-null    int64  
 10  beras           151 non-null    int64  
 11  biji_bijian     151 non-null    int64  
 12  buah            151 non-null    int64  
 13  daging_ayam     151 non-null    int64  
 14  daging_babi     151 non-null    int64  
 15  daging_kambing  151 non-null    int64  
 16  daging_sapi     151 non-null    int64  
 17  ikan            151 non-null    int

In [7]:
food_data.columns

Index(['food_id', 'nama_makanan', 'sumber', 'tipe', 'jenis_olahan', 'kalori',
       'protein', 'lemak', 'karbohidrat', 'daging_kerbau', 'beras',
       'biji_bijian', 'buah', 'daging_ayam', 'daging_babi', 'daging_kambing',
       'daging_sapi', 'ikan', 'kedelai', 'sayur', 'susu', 'telur_ayam',
       'tepung', 'umbi_umbian'],
      dtype='object')

In [8]:
food_data['nama_makanan'] = food_data['nama_makanan'].astype(str)

In [9]:
food_data_query = food_data[['food_id', 'nama_makanan', 'sumber', 'tipe', 'jenis_olahan', 'kalori',
       'daging_kerbau', 'beras', 'biji_bijian', 'buah', 'daging_ayam', 'daging_babi', 'daging_kambing',
       'daging_sapi', 'ikan', 'kedelai', 'sayur', 'susu', 'telur_ayam', 'tepung', 'umbi_umbian']]

In [10]:
column_food_used = ['nama_makanan']
food_data_used = food_data[column_food_used]

In [11]:
food_data_used.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nama_makanan  151 non-null    object
dtypes: object(1)
memory usage: 1.3+ KB


## **b. Food User Data**

In [12]:
user_data = pd.read_csv("dataset/dataset/user_data_final.csv")

In [13]:
user_data

,nama_makanan,gender,age,body_weight,body_height,cal_need
0,Bakso,1,22,54,160,1869
1,Bubur Ayam,1,22,54,160,1869
2,Mi Goreng,1,22,54,160,1869
3,Sate,1,22,54,160,1869
4,Soto,1,22,54,160,1869
...,...,...,...,...,...,...
22796,Kue Sus,0,29,61,157,1962
22797,Martabak Mesir,0,29,61,157,1962
22798,Mie Aceh Rebus,0,29,61,157,1962
22799,Mie Pangsit Basah,0,29,61,157,1962


In [14]:
user_data.columns

Index(['nama_makanan', 'gender', 'age', 'body_weight', 'body_height',
       'cal_need'],
      dtype='object')

In [15]:
user_data['nama_makanan'] = user_data['nama_makanan'].astype(str)
user_data['gender'] = user_data['gender'].astype(np.int32)
user_data['age'] = user_data['age'].astype(np.int32)
user_data['body_weight'] = user_data['body_weight'].astype(np.int32)
user_data['body_height'] = user_data['body_height'].astype(np.int32)
user_data['cal_need'] = user_data['cal_need'].astype(np.int32)

In [16]:
column_user_data = ['nama_makanan', 'gender', 'age', 'body_weight', 'body_height', 'cal_need']
user_data_used = user_data[column_user_data]

In [17]:
user_data_used

,nama_makanan,gender,age,body_weight,body_height,cal_need
0,Bakso,1,22,54,160,1869
1,Bubur Ayam,1,22,54,160,1869
2,Mi Goreng,1,22,54,160,1869
3,Sate,1,22,54,160,1869
4,Soto,1,22,54,160,1869
...,...,...,...,...,...,...
22796,Kue Sus,0,29,61,157,1962
22797,Martabak Mesir,0,29,61,157,1962
22798,Mie Aceh Rebus,0,29,61,157,1962
22799,Mie Pangsit Basah,0,29,61,157,1962


In [18]:
user_data_used.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22801 entries, 0 to 22800
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nama_makanan  22801 non-null  object
 1   gender        22801 non-null  int32 
 2   age           22801 non-null  int32 
 3   body_weight   22801 non-null  int32 
 4   body_height   22801 non-null  int32 
 5   cal_need      22801 non-null  int32 
dtypes: int32(5), object(1)
memory usage: 623.6+ KB


# **3. Building a Model**

In [19]:
# food_column_values = food_data_used['Nama Bahan Makanan'].tolist()
food_features = tf.data.Dataset.from_tensor_slices(dict(food_data_used))

In [20]:
food_features

<_TensorSliceDataset element_spec={'nama_makanan': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [21]:
food_features = food_features.map(lambda x: x["nama_makanan"])

In [22]:
food_features

<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [23]:
user_features = tf.data.Dataset.from_tensor_slices(dict(user_data_used))

In [24]:
user_features

<_TensorSliceDataset element_spec={'nama_makanan': TensorSpec(shape=(), dtype=tf.string, name=None), 'gender': TensorSpec(shape=(), dtype=tf.int32, name=None), 'age': TensorSpec(shape=(), dtype=tf.int32, name=None), 'body_weight': TensorSpec(shape=(), dtype=tf.int32, name=None), 'body_height': TensorSpec(shape=(), dtype=tf.int32, name=None), 'cal_need': TensorSpec(shape=(), dtype=tf.int32, name=None)}>

In [25]:
user_features = user_features.map(lambda x: {
    "nama_makanan": x["nama_makanan"],
    "gender": x["gender"],
    "age": x["age"],
    "body_weight": x["body_weight"],
    "body_height": x["body_height"],
    "cal_need" : x["cal_need"]
})

In [26]:
user_features

<_MapDataset element_spec={'nama_makanan': TensorSpec(shape=(), dtype=tf.string, name=None), 'gender': TensorSpec(shape=(), dtype=tf.int32, name=None), 'age': TensorSpec(shape=(), dtype=tf.int32, name=None), 'body_weight': TensorSpec(shape=(), dtype=tf.int32, name=None), 'body_height': TensorSpec(shape=(), dtype=tf.int32, name=None), 'cal_need': TensorSpec(shape=(), dtype=tf.int32, name=None)}>

## **A. User and Food Features**

In [27]:
unique_food_titles = np.unique(np.concatenate(list(food_features.batch(32))))

unique_user_gender = np.unique(np.concatenate(list(user_features.batch(32).map(
    lambda x: x["gender"]))))

unique_user_age = np.unique(np.concatenate(list(user_features.batch(32).map(
    lambda x: x["age"]))))

unique_user_weight = np.unique(np.concatenate(list(user_features.batch(32).map(
    lambda x: x["body_weight"]))))

unique_user_height = np.unique(np.concatenate(list(user_features.batch(32).map(
    lambda x: x["body_height"]))))

unique_user_calories = np.unique(np.concatenate(list(user_features.batch(1_000).map(
    lambda x: x["cal_need"]))))

## **B. Build Model**

In [28]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.gender_embedding = tf.keras.Sequential([
        tf.keras.layers.IntegerLookup(
            vocabulary=unique_user_gender, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_gender) + 1, 32),
    ])

    self.age_embedding = tf.keras.Sequential([
        tf.keras.layers.IntegerLookup(
            vocabulary=unique_user_age, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_age) + 1, 32),
    ])

    self.weight_embedding = tf.keras.Sequential([
        tf.keras.layers.IntegerLookup(
            vocabulary=unique_user_weight, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_weight) + 1, 32),
    ])

    self.height_embedding = tf.keras.Sequential([
        tf.keras.layers.IntegerLookup(
            vocabulary=unique_user_height, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_height) + 1, 32),
    ])

    self.calories_embedding = tf.keras.Sequential([
        tf.keras.layers.IntegerLookup(
            vocabulary=unique_user_calories, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_calories) + 1, 32),
    ])

  def call(self, inputs):
    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.gender_embedding(inputs["gender"]),
        self.age_embedding(inputs["age"]),
        self.weight_embedding(inputs["body_weight"]),
        self.height_embedding(inputs["body_height"]),
        self.calories_embedding(inputs["cal_need"])
    ], axis=1)


In [29]:
class QueryModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, layer_sizes):
    """Model for encoding user queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = UserModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [30]:
class FoodModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_food_titles,mask_token=None),
      tf.keras.layers.Embedding(len(unique_food_titles) + 1, 32)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.title_vectorizer.adapt(food_features)

  def call(self, titles):
    return tf.concat([
        self.title_embedding(titles),
        self.title_text_embedding(titles),
    ], axis=1)


In [31]:
class CandidateModel(tf.keras.Model):
  """Model for encoding foods."""

  def __init__(self, layer_sizes):
    """Model for encoding foods.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.embedding_model = FoodModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)


In [32]:
class FoodlensModel(tfrs.models.Model):

  def __init__(self, layer_sizes):
    super().__init__()
    self.query_model = QueryModel(layer_sizes)
    self.candidate_model = CandidateModel(layer_sizes)
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=food_features.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):

    query_embeddings = self.query_model({
        "gender": features["gender"],
        "age": features["age"],
        "body_weight": features["body_weight"],
        "body_height": features["body_height"],
        "cal_need": features["cal_need"],
    })
    food_embeddings = self.candidate_model(features["nama_makanan"])

    return self.task(
        query_embeddings, food_embeddings, compute_metrics=not training)


## **C. Training Model**

In [33]:
tf.random.set_seed(42)
shuffled = user_features.shuffle(22801, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(18241)
test = shuffled.skip(18241).take(4560)

cached_train = train.shuffle(18241).batch(2048)
cached_test = test.batch(4096).cache()

In [34]:
# set model checkpoint to record the weight during & after fitting
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('./checkpoint_folder/training_weights{epoch:02d}',save_weights_only=True,save_best_only=True)

In [35]:
num_epochs = 50

model = FoodlensModel([128, 64, 32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

one_layer_history = model.fit(
      cached_train,
      validation_data=cached_test,
      validation_freq=5,
      epochs=num_epochs,
      verbose=1,
      callbacks=[model_checkpoint]
    )

accuracy = one_layer_history.history["val_factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Top-100 accuracy: {accuracy:.2f}.")

Epoch 1/50
9/9 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15433.9576 - regularization_loss: 0.0000e+00 - total_loss: 15433.9576

9/9 [==============================] - 7s 186ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15288.5224 - regularization_loss: 0.0000e+00 - total_loss: 15288.5224
Epoch 2/50
9/9 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15433.8878 - regularization_loss: 0.0000e+00 - total_loss: 15433.8878

9/9 [==============================] - 2s 171ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15288.2241 - regularization_loss: 0.0000e+00 - total_loss: 15288.2241
Epoch 3/50
9/9 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15433.2667 - regularization_loss: 0.0000e+00 - total_loss: 15433.2667

9/9 [==============================] - 2s 198ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15287.6512 - regularization_loss: 0.0000e+00 - total_loss: 15287.6512
Epoch 4/50
9/9 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15433.2241 - regularization_loss: 0.0000e+00 - total_loss: 15433.2241

9/9 [==============================] - 3s 258ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15287.6139 - regularization_loss: 0.0000e+00 - total_loss: 15287.6139
Epoch 5/50
9/9 [==============================] - 5s 621ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15287.6105 - regularization_loss: 0.0000e+00 - total_loss: 15287.6105 - val_factorized_top_k/top_1_categorical_accuracy: 0.0033 - val_factorized_top_k/top_5_categorical_accuracy: 0.0254 - val_factorized_top_k/

...

9/9 [==============================] - 1s 144ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15285.8303 - regularization_loss: 0.0000e+00 - total_loss: 15285.8303
Epoch 45/50
9/9 [==============================] - 2s 151ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15285.5706 - regularization_loss: 0.0000e+00 - total_loss: 15285.5706 - val_factorized_top_k/top_1_categorical_accuracy: 0.0011 - val_factorized_top_k/top_5_categorical_accuracy: 0.0217 - val_factorized_top_k

9/9 [==============================] - 1s 90ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15285.7668 - regularization_loss: 0.0000e+00 - total_loss: 15285.7668
Epoch 47/50
9/9 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15430.9606 - regularization_loss: 0.0000e+00 - total_loss: 15430.9606

9/9 [==============================] - 1s 89ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15285.4700 - regularization_loss: 0.0000e+00 - total_loss: 15285.4700
Epoch 48/50
9/9 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15431.2524 - regularization_loss: 0.0000e+00 - total_loss: 15431.2524

9/9 [==============================] - 1s 92ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15285.5850 - regularization_loss: 0.0000e+00 - total_loss: 15285.5850
Epoch 49/50
9/9 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15430.1238 - regularization_loss: 0.0000e+00 - total_loss: 15430.1238

9/9 [==============================] - 1s 91ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15284.5220 - regularization_loss: 0.0000e+00 - total_loss: 15284.5220
Epoch 50/50
9/9 [==============================] - 2s 165ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15285.3692 - regularization_loss: 0.0000e+00 - total_loss: 15285.3692 - val_factorized_top_k/top_1_categorical_accuracy: 0.0018 - val_factorized_top_k/top_5_categorical_accuracy: 0.0184 - val_factorized_top_k/

## **C. Prediction**

In [36]:
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)
index.index_from_dataset(
  tf.data.Dataset.zip((food_features.batch(100), food_features.batch(100).map(model.candidate_model)))
)

_, titles = index({
    "gender": np.array([1]),
    "age": np.array([21]),
    "body_weight": np.array([75]),
    "body_height": np.array([167]),
    "cal_need": np.array([1681])},
    k=100
)

In [37]:
titles[0].numpy()

array([b'Soto madura', b'Keripik tempe', b'Wingko babat', b'Soto jeroan',
       b'Ikan Patin bakar', b'Fillet O-Fish', b'Langsat, Segar',
       b'Manggis, Segar', b'Dodol galamai', b'Soto pemalang',
       b'Gulai asam keueung', b'Soto sukaraja', b'Alpukat',
       b'Ikan Papuyu, Bakar', b'Belimbing', b'Telur mata sapi',
       b'Ayam goreng kalasan', b'Daging sapi kornet', b'Lapis legit',
       b'Wortel Rebus', b'Tempe kedelai murni, goreng', b'Gulai Kambing',
       b'Mie Aceh Rebus', b'Nanas, Segar', b'Melon, Segar', b'Jambu Biji',
       b'Lempog durian', b'Cap Cai, Sayur', b'Kembang tahu rebus',
       b'Kue gelang', b'Rambutan Segar', b'Sop kaki sapi',
       b'Sale pisang siam', b'Sop konro', b'Ketupat kandangan',
       b'Dodol manado', b'Sop saudara', b'Sale kesemek', b'Soto betawi',
       b'Teri balado', b'Cumi-cumi goreng', b'Dendeng sapi mentah',
       b'Pisang Ambon', b'Ledre pisang', b'Bihun Goreng Instan',
       b'Saboko sardin', b'Beef teriyaki', b'Soto kudus',
  

In [38]:
titles_array = [title.decode('utf-8') for title in titles[0].numpy()]

In [39]:
titles_array

['Soto madura',
 'Keripik tempe',
 'Wingko babat',
 'Soto jeroan',
 'Ikan Patin bakar',
 'Fillet O-Fish',
 'Langsat, Segar',
 'Manggis, Segar',
 'Dodol galamai',
 'Soto pemalang',
 'Gulai asam keueung',
 'Soto sukaraja',
 'Alpukat',
 'Ikan Papuyu, Bakar',
 'Belimbing',
 'Telur mata sapi',
 'Ayam goreng kalasan',
 'Daging sapi kornet',
 'Lapis legit',
 'Wortel Rebus',
 'Tempe kedelai murni, goreng',
 'Gulai Kambing',
 'Mie Aceh Rebus',
 'Nanas, Segar',
 'Melon, Segar',
 'Jambu Biji',
 'Lempog durian',
 'Cap Cai, Sayur',
 'Kembang tahu rebus',
 'Kue gelang',
 'Rambutan Segar',
 'Sop kaki sapi',
 'Sale pisang siam',
 'Sop konro',
 'Ketupat kandangan',
 'Dodol manado',
 'Sop saudara',
 'Sale kesemek',
 'Soto betawi',
 'Teri balado',
 'Cumi-cumi goreng',
 'Dendeng sapi mentah',
 'Pisang Ambon',
 'Ledre pisang',
 'Bihun Goreng Instan',
 'Saboko sardin',
 'Beef teriyaki',
 'Soto kudus',
 'Ikan Lele Goreng',
 'Dendeng Mujahir, Goreng',
 'Bika Ambon',
 'Kesemek, Segar',
 'Tekwan',
 'Sop buntut'

In [40]:
# allergies=['daging_babi', 'daging_kambing', 'sayur', 'daging_sapi']
# for allergy in allergies:
#     filtered_data_food = food_data_query[food_data_query[allergy] != 1]

# filtered_data_food

In [46]:
# WORK
def recommendation(list_food, food_query, user_calories, allergies):

    data_nut_food = food_query[food_query["nama_makanan"].isin(list_food)]

    data_nut_food = data_nut_food[data_nut_food['tipe'] != 'Makanan Ringan']
    list_food = data_nut_food['nama_makanan'].values

    if allergies == None:
        list_food = list_food[:15]
        filtered_data_food = data_nut_food[filtered_data_food['nama_makanan'].isin(list_food)]
    else:
        for allergy in allergies:
            filtered_data_food = data_nut_food[data_nut_food[allergy] != 1]

    list_food = filtered_data_food['nama_makanan'].values

    data_nut_cal = filtered_data_food[["nama_makanan", "kalori"]]
    comb_3 = combinations(list_food, 3)
    valid_combinations = []

    for comb in comb_3:
        total_calories = sum(data_nut_cal[data_nut_cal["nama_makanan"].isin(comb)]["kalori"])
        if total_calories <= user_calories:
            valid_combinations.append((comb, total_calories))

    valid_combinations.sort(key=lambda x: x[1], reverse=True)  # Sort by total calories

    return valid_combinations[:1]

In [47]:
hafidh = recommendation(titles_array, food_data_query, user_calories=1800, allergies=['daging_babi'])

In [48]:
hafidh

[(('Mi Goreng', 'Usus ayam goreng', 'Dendeng Mujahir, Goreng'), 1546)]

In [44]:
# # Bisa tapi lama banget

# def recommendation(list_food, food_query, user_calories, allergies):
#     num_eat=4
#     data_nut_food = food_query[food_query["nama_makanan"].isin(list_food)]

#     if allergies == None:
#         list_food = list_food[:15]
#         filtered_data_food = data_nut_food[filtered_data_food['nama_makanan'].isin(list_food)]
#     else:
#         for allergy in allergies:
#             filtered_data_food = data_nut_food[data_nut_food[allergy] != 1]


#     data_nut_cal = filtered_data_food[["nama_makanan", "kalori"]]

#     # Filter foods into snack and non-snack categories
#     snack_foods = filtered_data_food[filtered_data_food['tipe'] != 'Makanan Berat']
#     list_snack = snack_foods['nama_makanan'].values

#     non_snack_foods = filtered_data_food[filtered_data_food['tipe'] != 'Makanan Ringan']
#     list_non_snack = non_snack_foods['nama_makanan'].values

#     # Generate combinations of 3 snack foods and 1 non-snack food
#     comb_4 = []
#     for snack_comb in combinations(list_non_snack, 3):
#         for non_snack_comb in combinations(list_snack, 1):
#             comb_4.append(snack_comb + non_snack_comb)

#     # Calculate and filter valid combinations based on total calories
#     valid_combinations = []
#     for comb in comb_4:
#         total_calories = sum(data_nut_cal[data_nut_cal["nama_makanan"].isin(comb)]["kalori"])
#         if total_calories <= user_calories:
#             valid_combinations.append((comb, total_calories))

#     valid_combinations.sort(key=lambda x: x[1], reverse=True)  # Sort by total calories

#     return valid_combinations  # Return the top num_eat combinations

In [45]:
hafidh = recommendation(titles_array, food_data_query, user_calories=1800, allergies=['daging_babi'])

KeyboardInterrupt: ignored

In [ ]:
hafidh